# Reaction Cross Section Challenge

Notebook adapted from the ROSE challenges made by Daniel Odell and Pablo Giuliani: https://indico.cern.ch/event/1223721/contributions/5394829/

The purpose of this challenge is to extract information of a nucleus through the scattering cross section of particles that interact with it (in this case, neutrons). We will use an optical potential to model such interaction. 

The acompanying re-scaled Schrodinger equation is:
$$
\left[ -\frac{d^2}{ds^2} + \frac{\ell(\ell+1)}{s^2} + U(s,\omega,k) - 1 \right] \phi(s) = 0~,
$$


where

$$
s\equiv k r~,
$$

where $k$ is the center-of-mass momentum and $r$ is the relative separation, and

$$
U(s,\omega,k) \equiv \frac{2\mu}{\hbar k^2}V(s/k,\omega)~.
$$.

We use the optical potential:

$$
    \omega= \{V_v,W_v,W_d,R,a\},

$$


$$
    \begin{aligned}
      &V(r,\omega) =-V_v f_\text{WS}(r,R,a) - iW_v f_\text{WS}(r,R,a) \\
      &-i4a_dW_d \frac{d}{dr}f_\text{WS}(r,R,a),
    \end{aligned}
$$
where the Woods-Saxon term is defined as:
\begin{equation}
    f_\text{WS}(r,R,a) = \Big[1+\text{exp}\Big( \frac{r-R}{a}\Big)  \Big]^{-1}.
\end{equation}

It contains three terms, a real volume interaction proportional to $V_v$, an imaginary volume interaction proportional to $W_v$, and an imaginary surface interaction proportional to $W_d$.

The Schrodinger equation must be solver for various values of $l$ up to some value $L_\text{max}$. Every solution is then compared to the asymptotic solutuions to Schrodinger equation in the abscence of a short range potential:
$$
    \phi(s)_{s\rightarrow \infty} \propto e^{i\delta_\ell}\Big(\text{cos}\delta_\ell\  F_\ell(0,s)+\text{sin}\delta_\ell \ G_\ell(0,s) \Big).
$$

that allows us to extract the phase shifts from the following equations:
$$
 e^{2i\delta_\ell} =\frac{H^-_\ell(a)-aR_\ell H'^{-}_\ell(a)}{H^{+}_\ell(a)-aR_\ell H'^{+}_\ell(a)},
$$

with:

$$
H^{\pm}(s)\equiv G_\ell(0,s)\pm iF_\ell(0,s)
$$

and the matching value at the matching radius defined as:

$$
 R_\ell\equiv \frac{1}{a}\frac{\phi(a)}{\phi'(a)}.
$$

Once the phase shifts have been computed, we can obtain the amplitude:
$$
    f_n(\theta) = \frac{1}{2ik}\sum_{\ell=0}^{L_\text{max}}(2\ell+1)P_\ell(\text{cos}\theta)\big(e^{2i\delta_\ell}-1 \big),
$$

from which the differential cross section is defined:

$$
   \frac{d\sigma}{d\Omega}(\theta) = |f_n(\theta)|^2.
$$

Your tasks are to:

- Import and plot the cross section data for 40Ca + neutrons at the energy 14 MeV.
- Optimize the five model parameters to reproduce the data as close as possible. Be mindful on the order of the parameters used in the solver we give you below.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.special import spherical_jn


In [ ]:
A=40

hbarc=197.326

AMU = 931.494102 # MeV/c^2, Particle Data Group
MASS_N = 1.008665 * AMU # MeV/c^2 PDG
MASS_P = 1.007276 * AMU # MeV/c^2 PDG
B_40CA = 342.0522 # BMEX

MASS_40CA = 20*MASS_P + 20*MASS_N - B_40CA
MU = MASS_40CA * MASS_N / (MASS_40CA + MASS_N)

energy = 14 # center-of-mass scattering energy, fixed for this example
k = np.sqrt(2*MU*energy)/hbarc


l_max = 10
l_list=list(range(l_max+1))


In [ ]:
#Interaction

def wood_saxon(r, R, a):
    return 1/(1 + np.exp((r-R)/a))


def wood_saxon_prime(r, R, a):
    return -1/a * np.exp((r-R)/a) / (1 + np.exp((r-R)/a))**2


def optical_potential(r, theta):
    Vv, Wv, Wd, R, R, a = theta
    return -Vv * wood_saxon(r, R, a) - \
           1j*Wv * wood_saxon(r, R, a) - \
           4j*a*Wd * wood_saxon_prime(r, R, a)

potential_parameters=5



In [ ]:
#Good guesses for the parameters:

Vv0 = 45
Wv0 = 2
Wd0 = -7

R = 4
av = 0.6


In [ ]:
def Gamow_factor(l):

    if l == 0:
        return 1
    else:
        return 1 / (2*l + 1) * Gamow_factor(l-1)


In [ ]:
s_endpts = np.array([1e-4, 8*np.pi]) # dimensionless
initial_conditions = np.array([s_endpts[0], 1.0]) # initial phi(0) and phi'(0) conditions
s_mesh = np.linspace(*s_endpts, 2000)


def solve_se(theta,l):
    '''
    Solves the Schrödinger Equation for a specified set of MN potential parameters.
    '''
    result = solve_ivp(
        lambda s, phi: np.array([phi[1],
            (mn_potential_tilde(s, theta) + l*(l+1)/s**2 - 1.0) * phi[0]]),
        s_endpts, initial_conditions, rtol=1e-12, atol=1e-20, dense_output=True, method='DOP853'
    )
    return result.sol(s_mesh)[0]